# Pixel clustering notebook

In [1]:
# import required packages
from datetime import datetime as dt
import os
import subprocess

import feather
import json
from matplotlib import rc_file_defaults
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from ark.analysis import visualize
from ark.phenotyping import som_utils
from ark.utils import data_utils, io_utils, load_utils, plot_utils
from ark.utils.metacluster_remap_gui import MetaClusterData, MetaClusterGui, metaclusterdata_from_files

In [2]:
def cluster_pixels_short(fovs, channels, base_dir, data_dir='pixel_mat_data',
                   norm_vals_name='post_rowsum_chan_norm.feather',
                   weights_name='pixel_weights.feather',
                   pc_chan_avg_som_cluster_name='pixel_channel_avg_som_cluster.csv',
                   batch_size=5):
    """Uses trained weights to assign cluster labels on full pixel data
    Saves data with cluster labels to `cluster_dir`. Computes and saves the average channel
    expression across pixel SOM clusters.
    Args:
        fovs (list):
            The list of fovs to subset on
        channels (list):
            The list of channels to subset on
        base_dir (str):
            The path to the data directory
        data_dir (str):
            Name of the directory which contains the full preprocessed pixel data
        norm_vals_name (str):
            The name of the file with the 99.9% normalized values, created by `train_pixel_som`
        weights_name (str):
            The name of the weights file created by `train_pixel_som`
        pc_chan_avg_som_cluster_name (str):
            The name of the file to save the average channel expression across all SOM clusters
        batch_size (int):
            The number of FOVs to process in parallel
    """

    # define the paths to the data
    data_path = os.path.join(base_dir, data_dir)
    norm_vals_path = os.path.join(base_dir, norm_vals_name)
    weights_path = os.path.join(base_dir, weights_name)
    output_path = data_path + '_clustered'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # if path to the preprocessed directory does not exist
    if not os.path.exists(data_path):
        raise FileNotFoundError('Pixel data directory %s does not exist in base_dir %s' %
                                (data_dir, base_dir))

    # if path to the normalized values file does not exist
    if not os.path.exists(norm_vals_path):
        raise FileNotFoundError('Normalized values file %s does not exist in base_dir %s' %
                                (norm_vals_path, base_dir))
    

    # if path to the weights file does not exist
    if not os.path.exists(weights_path):
        raise FileNotFoundError('Weights file %s does not exist in base_dir %s' %
                                (weights_name, base_dir))

    # verify that all provided fovs exist in the folder
    # NOTE: remove the channel and pixel normalization files as those are not pixel data
    data_files = io_utils.list_files(data_path, substrs='.feather')
    misc_utils.verify_in_list(provided_fovs=fovs,
                              subsetted_fovs=io_utils.remove_file_extensions(data_files))

    weights = feather.read_dataframe(os.path.join(base_dir, weights_name))

    # ensure the norm vals columns and the FOV data contain valid indexes
    # ignoring metadata columns in the FOV data, the columns need to be in exactly
    # the same order across both datasets (normalized values and FOV values)
    norm_vals = feather.read_dataframe(os.path.join(base_dir, norm_vals_name))
    sample_fov = feather.read_dataframe(os.path.join(base_dir, data_dir, data_files[0]))

    # for verification purposes, drop the metadata columns
    cols_to_drop = ['fov', 'row_index', 'column_index']
    for col in ['segmentation_label', 'pixel_som_cluster',
                'pixel_meta_cluster', 'pixel_meta_cluster_rename']:
        if col in sample_fov.columns.values:
            cols_to_drop.append(col)

    sample_fov = sample_fov.drop(
        columns=cols_to_drop
    )
    misc_utils.verify_same_elements(
        enforce_order=True,
        norm_vals_columns=norm_vals.columns.values,
        pixel_data_columns=sample_fov.columns.values
    )

    # ensure the weights columns are valid indexes
    misc_utils.verify_same_elements(
        enforce_order=True,
        pixel_weights_columns=weights.columns.values,
        pixel_data_columns=sample_fov.columns.values
    )

    # run the trained SOM on the dataset, assigning clusters
    process_args = ['Rscript', '/run_pixel_som.R', ','.join(fovs),
                    data_path, norm_vals_path, weights_path, str(batch_size), output_path]

    process = subprocess.Popen(process_args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # continuously poll the process for output/error so it gets displayed in the Jupyter notebook
    while True:
        # convert from byte string
        output = process.stdout.readline().decode('utf-8')

        # if the output is nothing and the process is done, break
        if process.poll() is not None:
            break
        if output:
            print(output.strip())

    if process.returncode != 0:
        raise MemoryError(
            "Process terminated: you likely have a memory-related error. Try increasing "
            "your Docker memory limit."
        )

In [3]:
def pixel_consensus_cluster_short(fovs, channels, base_dir, max_k=20, cap=3,
                            data_dir='pixel_mat_data_clustered',
                            pc_chan_avg_som_cluster_name='pixel_channel_avg_som_cluster.csv',
                            pc_chan_avg_meta_cluster_name='pixel_channel_avg_meta_cluster.csv',
                            clust_to_meta_name='pixel_clust_to_meta.feather',
                            batch_size=5, seed=42):
    """Run consensus clustering algorithm on pixel-level summed data across channels
    Saves data with consensus cluster labels to `consensus_dir`. Computes and saves the
    average channel expression across pixel meta clusters. Assigns meta cluster labels
    to the data stored in `pc_chan_avg_som_cluster_name`.
    Args:
        fovs (list):
            The list of fovs to subset on
        channels (list):
            The list of channels to subset on
        base_dir (str):
            The path to the data directory
        max_k (int):
            The number of consensus clusters
        cap (int):
            z-score cap to use when hierarchical clustering
        data_dir (str):
            Name of the directory which contains the full preprocessed pixel data.
            This data should also have the SOM cluster labels appended from `cluster_pixels`.
        pc_chan_avg_som_cluster_name (str):
            Name of file to save the channel-averaged results across all SOM clusters to
        pc_chan_avg_meta_cluster_name (str):
            Name of file to save the channel-averaged results across all meta clusters to
        clust_to_meta_name (str):
            Name of file storing the SOM cluster to meta cluster mapping
        batch_size (int):
            The number of FOVs to process in parallel
        seed (int):
            The random seed to set for consensus clustering
    """

    # define the paths to the data
    data_path = os.path.join(base_dir, data_dir)
    som_cluster_avg_path = os.path.join(base_dir, pc_chan_avg_som_cluster_name)
    clust_to_meta_path = os.path.join(base_dir, clust_to_meta_name)
    output_path = data_path + '_consensus'
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # if the path to the SOM clustered data doesn't exist
    if not os.path.exists(data_path):
        raise FileNotFoundError(
            'Data dir %s does not exist in base_dir %s' %
            (data_dir, base_dir)
        )

    # if the path to the average channel expression per SOM cluster doesn't exist
    if not os.path.exists(som_cluster_avg_path):
        raise FileNotFoundError(
            'Channel avg per SOM cluster file %s does not exist in base_dir %s' %
            (pc_chan_avg_som_cluster_name, base_dir)
        )

    # run the consensus clustering process
    process_args = ['Rscript', '/pixel_consensus_cluster.R', ','.join(fovs), ','.join(channels),
                    str(max_k), str(cap), data_path, som_cluster_avg_path,
                    clust_to_meta_path, str(batch_size), str(seed), output_path]

    process = subprocess.Popen(process_args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # continuously poll the process for output/error so it gets displayed in the Jupyter notebook
    while True:
        # convert from byte string
        output = process.stdout.readline().decode('utf-8')

        # if the output is nothing and the process is done, break
        if process.poll() is not None:
            break
        if output:
            print(output.strip())

    if process.returncode != 0:
        raise MemoryError(
            "Process terminated: you likely have a memory-related error. Try increasing "
            "your Docker memory limit."
        )

In [4]:
def update_pixel_meta_labels(pixel_data_path, pixel_remapped_dict,
                             pixel_renamed_meta_dict, fov):
    """Helper function to reassign meta cluster names based on remapping scheme to a FOV
    Args:
        pixel_data_path (str):
            The path to the pixel data drectory
        pixel_remapped_dict (dict):
            The mapping from pixel SOM cluster to pixel meta cluster label (not renamed)
        pixel_renamed_meta_dict (dict):
            The mapping from pixel meta cluster label to renamed pixel meta cluster name
        fov (str):
            The name of the FOV to process
    """

    # get the path to the fov
    fov_path = os.path.join(pixel_data_path, fov + '.feather')

    # read in the fov data with SOM and meta cluster labels
    fov_data = feather.read_dataframe(fov_path)

    # ensure that no SOM clusters are missing from the mapping
    misc_utils.verify_in_list(
        fov_som_labels=fov_data['pixel_som_cluster'],
        som_labels_in_mapping=list(pixel_remapped_dict.keys())
    )

    # assign the new meta cluster labels
    fov_data['pixel_meta_cluster'] = fov_data['pixel_som_cluster'].map(
        pixel_remapped_dict
    )

    # assign the renamed meta cluster names
    fov_data['pixel_meta_cluster_rename'] = fov_data['pixel_meta_cluster'].map(
        pixel_renamed_meta_dict
    )

    # resave the data with the new meta cluster lables
    feather.write_dataframe(fov_data, fov_path, compression='uncompressed')


def apply_pixel_meta_cluster_remapping(fovs, channels, base_dir,
                                       pixel_data_dir,
                                       pixel_remapped_name,
                                       pc_chan_avg_som_cluster_name,
                                       pc_chan_avg_meta_cluster_name,
                                       batch_size=5):
    """Apply the meta cluster remapping to the data in `pixel_consensus_dir`.
    Resave the re-mapped consensus data to `pixel_consensus_dir` and re-runs the
    average channel expression per pixel meta cluster computation.
    Re-maps the pixel SOM clusters to meta clusters in `pc_chan_avg_som_cluster_name`.
    Args:
        fovs (list):
            The list of fovs to subset on
        channels (list):
            The list of channels to subset on
        base_dir (str):
            The path to the data directories
        pixel_data_dir (str):
            Name of directory with the full pixel data.
            This data should also have the SOM cluster labels appended from `cluster_pixels`
            and the meta cluster labels appended from `pixel_consensus_cluster`.
        pixel_remapped_name (str):
            Name of the file containing the pixel SOM clusters to their remapped meta clusters
        pc_chan_avg_som_cluster_name (str):
            Name of the file containing the channel-averaged results across all SOM clusters
        pc_chan_avg_meta_cluster_name (str):
            Name of the file containing the channel-averaged results across all meta clusters
        batch_size (int):
            The number of FOVs to process in parallel
    """

    # define the data paths
    pixel_data_path = os.path.join(base_dir, pixel_data_dir)
    pixel_remapped_path = os.path.join(base_dir, pixel_remapped_name)
    som_cluster_avg_path = os.path.join(base_dir, pc_chan_avg_som_cluster_name)
    meta_cluster_avg_path = os.path.join(base_dir, pc_chan_avg_meta_cluster_name)

    # file path validation
    if not os.path.exists(pixel_data_path):
        raise FileNotFoundError('Pixel data dir %s does not exist in base_dir %s' %
                                (pixel_data_dir, base_dir))

    if not os.path.exists(pixel_remapped_path):
        raise FileNotFoundError('Pixel remapping file %s does not exist in base_dir %s' %
                                (pixel_remapped_name, base_dir))

    if not os.path.exists(som_cluster_avg_path):
        raise FileNotFoundError(
            'Channel average per SOM cluster file %s does not exist in base_dir %s' %
            (pc_chan_avg_meta_cluster_name, base_dir))

    if not os.path.exists(meta_cluster_avg_path):
        raise FileNotFoundError(
            'Channel average per meta cluster file %s does not exist in base_dir %s' %
            (pc_chan_avg_meta_cluster_name, base_dir))

    # read in the remapping
    pixel_remapped_data = pd.read_csv(pixel_remapped_path)

    # assert the correct columns are contained
    misc_utils.verify_same_elements(
        remapped_data_cols=pixel_remapped_data.columns.values,
        required_cols=['cluster', 'metacluster', 'mc_name']
    )

    # rename columns in pixel_remapped_data so it plays better with the existing
    # pixel_som_cluster and pixel_meta_cluster
    pixel_remapped_data = pixel_remapped_data.rename(
        {
            'cluster': 'pixel_som_cluster',
            'metacluster': 'pixel_meta_cluster',
            'mc_name': 'pixel_meta_cluster_rename'
        },
        axis=1
    )

    # create the mapping from pixel SOM to pixel meta cluster
    pixel_remapped_dict = dict(
        pixel_remapped_data[
            ['pixel_som_cluster', 'pixel_meta_cluster']
        ].values
    )

    # create the mapping from pixel meta cluster to renamed pixel meta cluster
    pixel_renamed_meta_dict = dict(
        pixel_remapped_data[
            ['pixel_meta_cluster', 'pixel_meta_cluster_rename']
        ].drop_duplicates().values
    )

    # define the partial function to iterate over
    fov_data_func = partial(
        update_pixel_meta_labels, pixel_data_path,
        pixel_remapped_dict, pixel_renamed_meta_dict
    )

    # define the multiprocessing context
    with multiprocessing.get_context('spawn').Pool(batch_size) as fov_data_pool:
        # define variable to keep track of number of fovs processed
        fovs_processed = 0

        # asynchronously generate and save the pixel matrices per FOV
        print("Using re-mapping scheme to re-label pixel meta clusters")
        for fov_batch in [fovs[i:(i + batch_size)] for i in range(0, len(fovs), batch_size)]:
            # NOTE: we don't need a return value since we're just resaving
            # and not computing intermediate data frames
            fov_data_pool.map(fov_data_func, fov_batch)

            # update number of fovs processed
            fovs_processed += len(fov_batch)

            print("Processed %d fovs" % fovs_processed)

    # re-compute average channel expression for each pixel meta cluster
    # and the number of pixels per meta cluster, add renamed meta cluster column in
    print("Re-computing average channel expression across pixel meta clusters")
    pixel_channel_avg_meta_cluster = compute_pixel_cluster_channel_avg(
        fovs,
        channels,
        base_dir,
        'pixel_meta_cluster',
        pixel_data_dir,
        keep_count=True
    )
    pixel_channel_avg_meta_cluster['pixel_meta_cluster_rename'] = \
        pixel_channel_avg_meta_cluster['pixel_meta_cluster'].map(pixel_renamed_meta_dict)

    # re-save the pixel channel average meta cluster table
    pixel_channel_avg_meta_cluster.to_csv(meta_cluster_avg_path, index=False)

    # re-assign pixel meta cluster labels back to the pixel channel average som cluster table
    print("Re-assigning meta cluster column in pixel SOM cluster average channel expression table")
    pixel_channel_avg_som_cluster = pd.read_csv(som_cluster_avg_path)

    pixel_channel_avg_som_cluster['pixel_meta_cluster'] = \
        pixel_channel_avg_som_cluster['pixel_som_cluster'].map(pixel_remapped_dict)

    pixel_channel_avg_som_cluster['pixel_meta_cluster_rename'] = \
        pixel_channel_avg_som_cluster['pixel_meta_cluster'].map(pixel_renamed_meta_dict)

    # re-save the pixel channel average som cluster table
    pixel_channel_avg_som_cluster.to_csv(som_cluster_avg_path, index=False)


In [5]:
def apply_pixel_meta_cluster_remapping_short(fovs, channels, base_dir,
                                       pixel_data_dir,
                                       pixel_remapped_name,
                                       pc_chan_avg_som_cluster_name,
                                       pc_chan_avg_meta_cluster_name,
                                       batch_size=5):
    """Apply the meta cluster remapping to the data in `pixel_consensus_dir`.
    Resave the re-mapped consensus data to `pixel_consensus_dir` and re-runs the
    average channel expression per pixel meta cluster computation.
    Re-maps the pixel SOM clusters to meta clusters in `pc_chan_avg_som_cluster_name`.
    Args:
        fovs (list):
            The list of fovs to subset on
        channels (list):
            The list of channels to subset on
        base_dir (str):
            The path to the data directories
        pixel_data_dir (str):
            Name of directory with the full pixel data.
            This data should also have the SOM cluster labels appended from `cluster_pixels`
            and the meta cluster labels appended from `pixel_consensus_cluster`.
        pixel_remapped_name (str):
            Name of the file containing the pixel SOM clusters to their remapped meta clusters
        pc_chan_avg_som_cluster_name (str):
            Name of the file containing the channel-averaged results across all SOM clusters
        pc_chan_avg_meta_cluster_name (str):
            Name of the file containing the channel-averaged results across all meta clusters
        batch_size (int):
            The number of FOVs to process in parallel
    """

    # define the data paths
    pixel_data_path = os.path.join(base_dir, pixel_data_dir)
    pixel_remapped_path = os.path.join(base_dir, pixel_remapped_name)
    som_cluster_avg_path = os.path.join(base_dir, pc_chan_avg_som_cluster_name)
    meta_cluster_avg_path = os.path.join(base_dir, pc_chan_avg_meta_cluster_name)

    # file path validation
    if not os.path.exists(pixel_data_path):
        raise FileNotFoundError('Pixel data dir %s does not exist in base_dir %s' %
                                (pixel_data_dir, base_dir))

    if not os.path.exists(pixel_remapped_path):
        raise FileNotFoundError('Pixel remapping file %s does not exist in base_dir %s' %
                                (pixel_remapped_name, base_dir))

    if not os.path.exists(som_cluster_avg_path):
        raise FileNotFoundError(
            'Channel average per SOM cluster file %s does not exist in base_dir %s' %
            (pc_chan_avg_meta_cluster_name, base_dir))

    if not os.path.exists(meta_cluster_avg_path):
        raise FileNotFoundError(
            'Channel average per meta cluster file %s does not exist in base_dir %s' %
            (pc_chan_avg_meta_cluster_name, base_dir))

    # read in the remapping
    pixel_remapped_data = pd.read_csv(pixel_remapped_path)

    # assert the correct columns are contained
    misc_utils.verify_same_elements(
        remapped_data_cols=pixel_remapped_data.columns.values,
        required_cols=['cluster', 'metacluster', 'mc_name']
    )

    # rename columns in pixel_remapped_data so it plays better with the existing
    # pixel_som_cluster and pixel_meta_cluster
    pixel_remapped_data = pixel_remapped_data.rename(
        {
            'cluster': 'pixel_som_cluster',
            'metacluster': 'pixel_meta_cluster',
            'mc_name': 'pixel_meta_cluster_rename'
        },
        axis=1
    )

    # create the mapping from pixel SOM to pixel meta cluster
    pixel_remapped_dict = dict(
        pixel_remapped_data[
            ['pixel_som_cluster', 'pixel_meta_cluster']
        ].values
    )

    # create the mapping from pixel meta cluster to renamed pixel meta cluster
    pixel_renamed_meta_dict = dict(
        pixel_remapped_data[
            ['pixel_meta_cluster', 'pixel_meta_cluster_rename']
        ].drop_duplicates().values
    )

    # define the partial function to iterate over
    fov_data_func = partial(
        som_utils.update_pixel_meta_labels, pixel_data_path,
        pixel_remapped_dict, pixel_renamed_meta_dict
    )

    # define the multiprocessing context
    with multiprocessing.get_context('spawn').Pool(batch_size) as fov_data_pool:
        # define variable to keep track of number of fovs processed
        fovs_processed = 0

        # asynchronously generate and save the pixel matrices per FOV
        print("Using re-mapping scheme to re-label pixel meta clusters")
        for fov_batch in [fovs[i:(i + batch_size)] for i in range(0, len(fovs), batch_size)]:
            # NOTE: we don't need a return value since we're just resaving
            # and not computing intermediate data frames
            fov_data_pool.map(fov_data_func, fov_batch)

            # update number of fovs processed
            fovs_processed += len(fov_batch)

            print("Processed %d fovs" % fovs_processed)

In [6]:
from functools import partial
import multiprocessing
import os
import json
import subprocess
import warnings

import feather
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import scipy.ndimage as ndimage
import scipy.stats as stats
from skimage.io import imread, imsave
import xarray as xr

from ark.analysis import visualize
import ark.settings as settings
from ark.utils import io_utils
from ark.utils import load_utils
from ark.utils import misc_utils

def compute_pixel_cluster_channel_avg(fovs, channels, base_dir, pixel_cluster_col,
                                      pixel_data_dir='pixel_mat_data', keep_count=False):
    """Compute the average channel values across each pixel SOM cluster
    Args:
        fovs (list):
            The list of fovs to subset on
        channels (list):
            The list of channels to subset on
        base_dir (str):
            The path to the data directories
        pixel_cluster_col (str):
            Name of the column to group by
        pixel_data_dir (str):
            Name of the directory containing the pixel data with cluster labels
        keep_count (bool):
            Whether to keep the count column when aggregating or not
            This should only be set to `True` for visualization purposes
    Returns:
        pandas.DataFrame:
            Contains the average channel values for each pixel SOM/meta cluster
    """

    # verify the pixel cluster col specified is valid
    misc_utils.verify_in_list(
        provided_cluster_col=[pixel_cluster_col],
        valid_cluster_cols=['pixel_som_cluster', 'pixel_meta_cluster']
    )

    # define the cluster averages DataFrame
    cluster_avgs = pd.DataFrame()
    bad_fovs = []

    for fov in fovs:
        # read in the fovs data
        try:
            fov_pixel_data = feather.read_dataframe(
                os.path.join(base_dir, pixel_data_dir, fov + '.feather')
            )

            # aggregate the sums and counts
            sum_by_cluster = fov_pixel_data.groupby(
                pixel_cluster_col
            )[channels].sum()
            count_by_cluster = fov_pixel_data.groupby(
                pixel_cluster_col
            )[channels].size().to_frame('count')

            # merge the results by column
            agg_results = pd.merge(
                sum_by_cluster, count_by_cluster, left_index=True, right_index=True
            ).reset_index()

            # concat the results together
            cluster_avgs = pd.concat([cluster_avgs, agg_results])
        except Exception as inst:
            print(inst)
            print(fov)
            bad_fovs.append(fov)
            

    # reset the index of cluster_avgs for consistency
    cluster_avgs = cluster_avgs.reset_index(drop=True)

    # sum the counts and the channel sums
    sum_count_totals = cluster_avgs.groupby(
        pixel_cluster_col
    )[channels + ['count']].sum().reset_index()

    # now compute the means using the count column
    sum_count_totals[channels] = sum_count_totals[channels].div(sum_count_totals['count'], axis=0)

    # convert cluster column to integer type
    sum_count_totals[pixel_cluster_col] = sum_count_totals[pixel_cluster_col].astype(int)

    # sort cluster col in ascending order
    sum_count_totals = sum_count_totals.sort_values(by=pixel_cluster_col)

    # drop the count column if specified
    if not keep_count:
        sum_count_totals = sum_count_totals.drop('count', axis=1)

    return sum_count_totals, bad_fovs

## 0: set file paths and parameters

* `base_dir`: the path to all of your imaging data. Should contain a directory for your images, segmentations, and cell table (generated from `Segment_Image_Data.ipynb`). This directory will also store all of the directories/files created during pixel clustering.
* `tiff_dir`: the path to the directory containing your imaging data
* `img_sub_folder`: if `tiff_dir` contains an additional subfolder structure, override `None` with the appropriate value
* `segmentation_dir`: the path to the directory containing your segmentations (generated from `Segment_Image_Data.ipynb`). Set this argument to `None` if you do not have segmentation labels or wish to run pixel clustering without them. However, note that you will not be able to run cell clustering as that process is heavily dependent on them.
* `seg_suffix`: the suffix plus the file extension of the segmented images for each FOV. Note that these should be the same for all FOVs and that normally, the value should be `'_feature_0.tif'`. This argument will be ignored if `segmentation_dir` is set to `None`
* `MIBItiff`: if the images in `tiff_dir` are mibitiff or not
* `mibitiff_suffix` (required if `MIBItiff` is True): the file suffix all mibitiff images contain

In [7]:
base_dir = "../data/external/"
tiff_dir = os.path.join(base_dir, 'image_data/samples')
img_sub_folder = None
segmentation_dir = os.path.join(base_dir, 'segmentation_data/deepcell_output')
seg_suffix = '_feature_0.tif'
MIBItiff = False
mibitiff_suffix = '-MassCorrected-Filtered.tiff'

* `fovs` (optional): set a specific set of fovs to load, default loads all the fovs in `tiff_dir`

In [8]:
# either get all fovs in the folder...
if MIBItiff:
    fovs = io_utils.list_files(tiff_dir, substrs=MIBItiff_suffix)
else:
    fovs = io_utils.list_folders(tiff_dir)

# ... or optionally, select a specific set of fovs manually
# fovs = ["fov14"]

In [9]:
test_fovs = ['TONIC_TMA23_R6C1',
 'TONIC_TMA15_R11C2',
 'TONIC_TMA23_R9C4',
 'TONIC_TMA6_R7C4',
 'TONIC_TMA23_R6C3',
 'TONIC_TMA18_R2C2',
 'TONIC_TMA19_R6C6',
 'TONIC_TMA18_R9C1',
 'TONIC_TMA4_R1C2',
 'TONIC_TMA23_R1C2',
 'TONIC_TMA10_R4C1',
 'TONIC_TMA10_R3C6',
 'TONIC_TMA1_R8C1',
 'TONIC_TMA6_R3C6',
 'TONIC_TMA14_R3C4',
 'TONIC_TMA21_R4C1',
 'TONIC_TMA13_R5C1',
 'TONIC_TMA7_R7C3',
 'TONIC_TMA12_R3C4',
 'TONIC_TMA6_R5C2',
 'TONIC_TMA11_R8C5',
 'TONIC_TMA6_R10C4',
 'TONIC_TMA16_R6C6',
 'TONIC_TMA20_R4C6',
 'TONIC_TMA22_R2C4',
 'TONIC_TMA9_R4C4',
 'TONIC_TMA3_R6C3',
 'TONIC_TMA16_R10C6',
 'TONIC_TMA3_R1C6',
 'TONIC_TMA14_R10C6',
 'TONIC_TMA12_R2C1',
 'TONIC_TMA10_R9C2',
 'TONIC_TMA12_R3C6',
 'TONIC_TMA17_R2C4',
 'TONIC_TMA2_R11C4',
 'TONIC_TMA2_R5C2',
 'TONIC_TMA24_R9C1',
 'TONIC_TMA3_R8C1',
 'TONIC_TMA4_R10C4',
 'TONIC_TMA8_R6C5',
 'TONIC_TMA4_R7C2',
 'TONIC_TMA20_R5C4',
 'TONIC_TMA17_R12C2',
 'TONIC_TMA10_R7C2',
 'TONIC_TMA8_R8C4',
 'TONIC_TMA3_R10C3',
 'TONIC_TMA23_R6C2',
 'TONIC_TMA24_R2C1',
 'TONIC_TMA9_R10C5',
 'TONIC_TMA8_R11C4']

## 1: Preprocess

Set a prefix to be applied to all data directories/files created by pixel and cell clustering. If the prefix is not set, a default of the datetime at the start of the run is used.

In [12]:
# explicitly set pixel_cluster_prefix to override datetime default
pixel_cluster_prefix = '20220707_full_cohort'

if pixel_cluster_prefix is None:
    pixel_cluster_prefix = dt.now().strftime('%Y-%m-%dT%H:%M:%S')

The following data directories/files will be created for preprocessing with names prefixed by `pixel_cluster_prefix`:

* `pixel_output_dir`: the name of the folder to store the pixel clustering directories/files
* `preprocessed_dir`: the name of the directory to save the preprocessed pixel data
* `subsetted_dir`: the name of the directory to save the subsetted pixel data
* `norm_vals_name`: file name to store the values used to normalize each channel on the full preprocessed dataset when assigning pixel SOM cluster labels.

In [13]:
# define the base output pixel folder using the specified pixel cluster prefix
pixel_output_dir = '%s_pixel_output_dir' % pixel_cluster_prefix
if not os.path.exists(os.path.join(base_dir, pixel_output_dir)):
    os.mkdir(os.path.join(base_dir, pixel_output_dir))

# define the preprocessed pixel data folders
pixel_data_dir = os.path.join(pixel_output_dir, '%s_pixel_mat_data' % pixel_cluster_prefix)
pixel_subset_dir = os.path.join(pixel_output_dir, '%s_pixel_mat_subset' % pixel_cluster_prefix)
norm_vals_name = os.path.join(pixel_output_dir, 'post_rowsum_chan_norm.feather')

For certain channels, such as membraneous tumor markers, the subcellular localization of the marker isn't important. Instead, what matters is that cells which are positive for the marker show up as positive. In these cases, we have sometimes found it useful to add additional blurring to these markers before clustering. This ensures that more of the pixels within the cell are positive for the marker, instead of only a few pixels at the border, especially for cells which are under-segmented. However, higher blur levels will also cause more of the pixels in neighboring cells to show up as positive. Therefore, this works best when you have other, robust markers (like CD45) which you can use to determine which cells are false positives for the blurred channel. If you have markers in your panel which fit this description, you can add them in the cell below. Then, when specifying the list of markers to include clustering, make sure to add `marker_name_smoothed`, as that is what the tiff will be called

In [8]:
# set an optional list of markers for additional blurring
blurred_channels = ['ECAD', 'CK17']
smooth_vals = 6
som_utils.smooth_channels(fovs=fovs, tiff_dir=tiff_dir, img_sub_folder=img_sub_folder, channels=blurred_channels, smooth_vals=smooth_vals)

/usr/local/lib/python3.6/site-packages/ark/utils/load_utils.py:174: UserWarning: The supplied non-float dtype int16 was overwritten to float64, because the loaded images are floats
  warnings.warn(f"The supplied non-float dtype {dtype} was overwritten to {data_dtype}, "


In [18]:
from ark.utils.load_utils import load_imgs_from_tree, load_imgs_from_dir
from ark.utils.misc_utils import verify_same_elements
import skimage.io as io

def filter_with_nuclear_mask(fovs, tiff_dir, segmentation_dir, channel, img_sub_folder=None, exclude=True):
    # convert to path-compatible format
    if img_sub_folder is None:
        img_sub_folder = ''
        
    
    for fov in fovs:
        img = load_utils.load_imgs_from_tree(data_dir=tiff_dir, img_sub_folder=img_sub_folder,
                                             fovs=[fov], channels=[channel]).values[0, :, :, 0]
        seg_img = io.imread(os.path.join(segmentation_dir, fov + '_feature_1.tif'))[0, ...]
        
        if exclude:
            suffix = '_nuc_exclude.tiff'
            seg_mask = seg_img > 0
        else:
            suffix = '_nuc_include.tiff'
            seg_mask = seg_img == 0
            
        img[seg_mask] = 0
        io.imsave(os.path.join(tiff_dir, fov, img_sub_folder, channel + suffix), img, check_contrast=False)

In [21]:
filter_with_nuclear_mask(fovs=fovs[10:], tiff_dir=tiff_dir, segmentation_dir=segmentation_dir, channel='CD11c')
filter_with_nuclear_mask(fovs=fovs, tiff_dir=tiff_dir, segmentation_dir=segmentation_dir, channel='FOXP3', exclude=False)

In [22]:
filter_with_nuclear_mask(fovs=fovs, tiff_dir=tiff_dir, segmentation_dir=segmentation_dir, channel='FOXP3', exclude=False)

Set the following arguments

* `channels`: set a subset to run pixel clustering over
* `blur_factor`: the sigma to use for the Gaussian filter when running the Gaussian blur. Higher values are more aggressive in smoothing signal.
* `subset_proportion`: the fraction of pixels to take from each fov. Sampling is random.

In [14]:
channels = ["CD45", "SMA", "Vim", "FAP", "Fibronectin", "Collagen1", "CK17_smoothed", "ECAD_smoothed", "ChyTr",
            "Calprotectin",  "CD3", "CD4", "CD8",  "CD11c_nuc_exclude", "CD14","CD20", "CD31", "CD56",  "CD68", "CD163", "HLADR", "FOXP3_nuc_include"]
blur_factor = 2
subset_proportion = 0.01

During pixel preprocessing, the following is done for each FOV:

* Gaussian blur each channel separately
* Remove empty pixels
* For the remaining pixels, normalize each channel by the sum of all the channels, this creates the full preprocessed dataset
* Subset a `subset_proportion` fraction of non-empty, normalized pixels, this creates the subsetted dataset

Note: if you get integer overflow errors loading in your data, try changing the `dtype` argument to a larger type.

In [67]:
# run pixel data preprocessing
som_utils.create_pixel_matrix(
    redo_fovs,
    channels,
    base_dir,
    tiff_dir,
    segmentation_dir,
    img_sub_folder=img_sub_folder,
    seg_suffix=seg_suffix,
    data_dir=pixel_data_dir,
    subset_dir=pixel_subset_dir,
    norm_vals_name=norm_vals_name,
    is_mibitiff=MIBItiff,
    blur_factor=blur_factor,
    subset_proportion=subset_proportion,
    dtype="float32", 
    pixel_output_dir=pixel_output_dir,
    pixel_cluster_prefix=pixel_cluster_prefix
)

Processed 2 fovs


## 2: pixel clustering

### 2.1: train pixel SOM

The following data directories/files will be created for pixel clustering:

* `pixel_weights_name`: file name to place the pixel SOM weights
* `pixel_som_to_meta_name`: file name to store the mapping between pixel SOM clusters and pixel meta clusters
* `pc_chan_avg_som_cluster_name`: file name to store the average channel expression across all pixel SOM clusters
* `pc_chan_avg_meta_cluster_name`: same as above except for pixel meta clusters
* `pixel_meta_cluster_remap_name`: for the meta cluster remapping process, the file to store the new SOM to meta mappings

In [15]:
pixel_weights_name = os.path.join(pixel_output_dir, '%s_pixel_weights.feather' % pixel_cluster_prefix)
pixel_som_to_meta_name = os.path.join(pixel_output_dir, '%s_pixel_som_to_meta.feather' % pixel_cluster_prefix)
pc_chan_avg_som_cluster_name = os.path.join(pixel_output_dir, '%s_pixel_channel_avg_som_cluster.csv' % pixel_cluster_prefix)
pc_chan_avg_meta_cluster_name = os.path.join(pixel_output_dir, '%s_pixel_channel_avg_meta_cluster.csv' % pixel_cluster_prefix)
pixel_meta_cluster_remap_name = os.path.join(pixel_output_dir, '%s_pixel_meta_cluster_mapping.csv' % pixel_cluster_prefix)

Train the pixel SOM on the subsetted data. Training is done using the `FlowSOM` algorithm.

Note that each channel is normalized by their 99.9% value across the entire subsetted dataset before training. These values get saved to `norm_vals_name`.

For a full set of parameters you can customize for `train_pixel_som`, please consult: <a href=https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.phenotyping.html#ark.phenotyping.som_utils.train_pixel_som>pixel training docs</a>.

In [60]:
# create the pixel-level SOM weights
som_utils.train_pixel_som(
    fovs,
    channels,
    base_dir,
    subset_dir=pixel_subset_dir,
    norm_vals_name=norm_vals_name,
    weights_name=pixel_weights_name,
    num_passes=1,
    xdim=17,
    ydim=17
)

Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.

Attaching package: ‘arrow’

The following object is masked from ‘package:utils’:

timestamp

Loading required package: igraph

Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

decompose, spectrum

The following object is masked from ‘package:base’:

union

Thanks for using FlowSOM. From version 2.1.4 on, the scale
parameter in the FlowSOM function defaults to FALSE
[1] "Reading the subsetted pixel matrix data"
[1] "Performing 99.9% normalization"
[1] "Training the SOM"
[1] "Save trained weights"


### 2.2: Assign pixel SOM clusters

Use the SOM weights learned from `train_pixel_som` to assign pixel clusters to the full preprocessed dataset.

Note that each channel is normalized by the respective value stored in `norm_vals_name` (computed in `train_pixel_som`) prior to cluster assignment.

This function also computes the average channel expression across all pixel SOM clusters as well as the number of pixels in each pixel SOM cluster (the data placed in `pc_chan_avg_som_cluster_name`). This is needed for pixel consensus clustering.

In [68]:
# use pixel SOM weights to assign pixel clusters
cluster_pixels_short(
    redo_fovs,
    channels,
    base_dir,
    data_dir=pixel_data_dir,
    norm_vals_name=norm_vals_name,
    weights_name=pixel_weights_name,
    pc_chan_avg_som_cluster_name=pc_chan_avg_som_cluster_name
)

Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.

Attaching package: ‘arrow’

The following object is masked from ‘package:utils’:

timestamp

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Loading required package: igraph

Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

decompose, spectrum

The following object is masked from ‘package:base’:

union

Thanks for using FlowSOM. From version 2.1.4 on, the scale
parameter in the FlowSOM function defaults to FALSE
[1] "Mapping pixel data to SOM cluster labels"
[1] "Processed 2 fovs"
[1] "Done!"


In [45]:
pixel_channel_avg_som_cluster, bad_fovs_cluster = compute_pixel_cluster_channel_avg(
        fovs,
        channels,
        base_dir,
        'pixel_som_cluster',
        pixel_data_dir + '_clustered',
        keep_count=True
    )

# save pixel_channel_avg_som_cluster
pixel_channel_avg_som_cluster.to_csv(
    os.path.join(base_dir, pc_chan_avg_som_cluster_name),
    index=False
)

Lz4 compressed input contains more than one frame
TONIC_TMA15_R2C6


In [66]:
redo_fovs = ['TONIC_TMA24_R5C2', 'TONIC_TMA15_R2C6']

In [20]:
# reprocess FOVs that fail
# TODO: delete before reprocessing
if len(bad_fovs_cluster) > 0:
    for fov in bad_fovs_cluster:
        os.remove(os.path.join(pixel_data_dir, fov + '.feather'))
        
    som_utils.create_pixel_matrix(
        bad_fovs_cluster,
        channels,
        base_dir,
        tiff_dir,
        segmentation_dir,
        img_sub_folder=img_sub_folder,
        seg_suffix=seg_suffix,
        data_dir=pixel_data_dir,
        subset_dir=pixel_subset_dir,
        norm_vals_name=norm_vals_name,
        is_mibitiff=MIBItiff,
        blur_factor=blur_factor,
        subset_proportion=subset_proportion,
        dtype="float32", 
        pixel_output_dir=pixel_output_dir
    )

    cluster_pixels_short(
        bad_fovs_cluster,
        channels,
        base_dir,
        data_dir=pixel_data_dir,
        norm_vals_name=norm_vals_name,
        weights_name=pixel_weights_name,
        pc_chan_avg_som_cluster_name=pc_chan_avg_som_cluster_name
    )

In [ ]:
# check if any of the bad_fovs failed again
bad_fovs_cluster_x1 = []
for fov in bad_fovs_cluster:
    try:
        feather_file = feather.read_dataframe(os.path.join(base_dir, pixel_data_dir, fov + '.feather'))
    except Exception as inst:
        print(fov)
        print(inst)
        bad_fovs_cluster_x1.append(fov)
print(bad_fovs_cluster_x1)

### 2.3: run pixel consensus clustering

With the SOM cluster labels assigned to the full preprocessed pixel data, assign consensus cluster labels. The consensus clusters are trained on the average channel expression across all pixel SOM clusters (the data stored in `pc_chan_avg_som_cluster_name`). These values are z-scored and capped at the value specified in the `cap` argument prior to training: this helps improve the meta clustering process.

After consensus clustering, the following are also computed:

* The average channel expression across all pixel meta clusters, and the number of pixels per meta cluster (the data placed in `pc_chan_avg_meta_cluster_name`)
* The meta cluster mapping for each pixel SOM cluster in `pc_chan_avg_som_cluster_name` (data is resaved, same data except with an associated meta cluster column)

For a full set of parameters you can customize for `pixel_consensus_cluster`, please consult: <a href=https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.phenotyping.html#ark.phenotyping.som_utils.pixel_consensus_cluster>pixel consensus clustering docs</a>

* `max_k`: the number of consensus clusters desired
* `cap`: used to clip z-scored values prior to consensus clustering (in the range `[-cap, cap]`)

In [69]:
max_k = 30
cap = 3

# run hierarchical clustering based on pixel SOM cluster assignments
pixel_consensus_cluster_short(
    redo_fovs,
    channels,
    base_dir,
    max_k=max_k,
    cap=cap,
    data_dir=pixel_data_dir + '_clustered',
    pc_chan_avg_som_cluster_name=pc_chan_avg_som_cluster_name,
    pc_chan_avg_meta_cluster_name=pc_chan_avg_meta_cluster_name,
    clust_to_meta_name=pixel_som_to_meta_name
)

Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.

Attaching package: ‘arrow’

The following object is masked from ‘package:utils’:

timestamp

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
[1] "Reading cluster averaged data"
[1] "Running consensus clustering"
[1] "Mapping pixel data to consensus cluster labels"
[1] "Processed 2 fovs"
[1] "Writing SOM to meta cluster mapping table"


In [16]:
## If the above fails
clust_to_meta_name=pixel_som_to_meta_name

# define the paths to the data
data_path = os.path.join(base_dir, pixel_data_dir)
som_cluster_avg_path = os.path.join(base_dir, pc_chan_avg_som_cluster_name)
clust_to_meta_path = os.path.join(base_dir, clust_to_meta_name)   

print("Computing average channel expression across pixel meta clusters")
pixel_channel_avg_meta_cluster, bad_fovs_meta = compute_pixel_cluster_channel_avg(
    fovs,
    channels,
    base_dir,
    'pixel_meta_cluster',
    pixel_data_dir + '_clustered_consensus',
    keep_count=True
)

# save pixel_channel_avg_meta_cluster
pixel_channel_avg_meta_cluster.to_csv(
    os.path.join(base_dir, pc_chan_avg_meta_cluster_name),
    index=False
)

# read in the clust_to_meta_name file
print("Mapping meta cluster values onto average channel expression across pixel SOM clusters")
som_to_meta_data = feather.read_dataframe(
    os.path.join(base_dir, clust_to_meta_name)
).astype(np.int64)

# read in the channel-averaged results across all pixel SOM clusters
pixel_channel_avg_som_cluster = pd.read_csv(som_cluster_avg_path)

# merge metacluster assignments in
pixel_channel_avg_som_cluster = pd.merge_asof(
    pixel_channel_avg_som_cluster, som_to_meta_data, on='pixel_som_cluster'
)

# resave channel-averaged results across all pixel SOM clusters with metacluster assignments
pixel_channel_avg_som_cluster.to_csv(
    som_cluster_avg_path,
    index=False
)

Computing average channel expression across pixel meta clusters
LZ4 decompress failed: ERROR_GENERIC
TONIC_TMA24_R5C2
Mapping meta cluster values onto average channel expression across pixel SOM clusters


In [25]:
bad_fovs_meta

[]

## 3: visualize results

### 3.1: use the interactive reclustering results to relabel pixel meta clusters

The visualization shows the z-scored average channel expression per pixel SOM and meta cluster. The heatmaps are faceted by pixel SOM clusters on the left and pixel meta clusters on the right.

## Usage

### Quickstart
- **Select**: Left Click
- **Remap**: **New metacluster** button or Right Click
- **Edit Metacluster Name**: Textbox at bottom right of the heatmaps.

### Selection and Remapping details
- To select a SOM cluster, click on its respective position in the **selected** bar. Click on it again to deselect.
- To select a meta cluster, click on its corresponding color in the **metacluster** bar. Click on it again to deselect.
- To remap the selected clusters, click the **New metacluster** button (alternatively, right click anywhere). Note that remapping an entire metacluster deletes it.
- To clear the selected SOM/meta clusters, use the **Clear Selection** button.
- **After remapping a meta cluster, make sure to deselect the newly created one to prevent unwanted combinations.**

### Other features and notes
- You will likely need to zoom out to see the entire visualization. To toggle Zoom, use Ctrl -/Ctrl + on Windows or ⌘ +/⌘ - on Mac.
- The bars at the top show the number of pixels in each SOM cluster.
- The text box at the bottom right allows you to rename a particular meta cluster. This can be useful as remapping may cause inconsistent numbering.
- Adjust the z-score limit using the slider on the bottom left to adjust your dynamic range.
- When meta clusters are combined or a meta cluster is renamed, the change is immediately saved to `pixel_meta_cluster_remap_name`.
- You won't be able to advance in the notebook until you've clicked `New metacluster` or renamed a meta cluster at least once. If you don't want to make changes, just click `New metacluster` to trigger a save before continuing.

The visualization shows the z-scored average channel expression per pixel SOM and meta cluster. The heatmaps are faceted by pixel SOM clusters on the left and pixel meta clusters on the right.

## Usage

### Quickstart
- **Select**: Left Click
- **Remap**: **New metacluster** button or Right Click
- **Edit Metacluster Name**: Textbox at bottom right of the heatmaps.

### Selection and Remapping details
- To select a SOM cluster, click on its respective position in the **selected** bar. Click on it again to deselect.
- To select a meta cluster, click on its corresponding color in the **metacluster** bar. Click on it again to deselect.
- To remap the selected clusters, click the **New metacluster** button (alternatively, right click anywhere). Note that remapping an entire metacluster deletes it.
- To clear the selected SOM/meta clusters, use the **Clear Selection** button.
- **After remapping a meta cluster, make sure to deselect the newly created one to prevent unwanted combinations.**

### Other features and notes
- You will likely need to zoom out to see the entire visualization. To toggle Zoom, use Ctrl -/Ctrl + on Windows or ⌘ +/⌘ - on Mac.
- The bars at the top show the number of pixels in each SOM cluster.
- The text box at the bottom right allows you to rename a particular meta cluster. This can be useful as remapping may cause inconsistent numbering.
- Adjust the z-score limit using the slider on the bottom left to adjust your dynamic range.
- When meta clusters are combined or a meta cluster is renamed, the change is immediately saved to `pixel_meta_cluster_remap_name`.
- You won't be able to advance in the notebook until you've clicked `New metacluster` or renamed a meta cluster at least once. If you don't want to make changes, just click `New metacluster` to trigger a save before continuing.

In [63]:
%matplotlib widget
rc_file_defaults()
plt.ion()

pixel_mcd = metaclusterdata_from_files(
    os.path.join(base_dir, pc_chan_avg_som_cluster_name),
    cluster_type='pixel'
)
pixel_mcd.output_mapping_filename = os.path.join(base_dir, pixel_meta_cluster_remap_name)
pixel_mcg = MetaClusterGui(pixel_mcd, width=17)

Relabel the pixel meta clusters using the mapping.

In [ ]:
som_utils.apply_pixel_meta_cluster_remapping(
    fovs[1210:],
    channels,
    base_dir,
    pixel_data_dir + '_clustered_consensus',
    pixel_meta_cluster_remap_name,
    pc_chan_avg_som_cluster_name,
    pc_chan_avg_meta_cluster_name, 
    pixel_data_dir + '_remapped'
)

In [71]:
fovs[1210:1215]

['TONIC_TMA19_R2C5',
 'TONIC_TMA19_R7C3',
 'TONIC_TMA19_R7C6',
 'TONIC_TMA19_R9C2',
 'TONIC_TMA19_R10C4']

In [74]:
# second half of apply_pixel_meta_cluster_remapping that updates files, since default function doesn't use updated directory

from ark.utils import misc_utils
pixel_data_path = os.path.join(base_dir, pixel_data_dir)
pixel_remapped_path = os.path.join(base_dir, pixel_meta_cluster_remap_name)
som_cluster_avg_path = os.path.join(base_dir, pc_chan_avg_som_cluster_name)
meta_cluster_avg_path = os.path.join(base_dir, pc_chan_avg_meta_cluster_name)

pixel_remapped_data = pd.read_csv(pixel_remapped_path)

# assert the correct columns are contained
misc_utils.verify_same_elements(
    remapped_data_cols=pixel_remapped_data.columns.values,
    required_cols=['cluster', 'metacluster', 'mc_name']
)

# rename columns in pixel_remapped_data so it plays better with the existing
# pixel_som_cluster and pixel_meta_cluster
pixel_remapped_data = pixel_remapped_data.rename(
    {
        'cluster': 'pixel_som_cluster',
        'metacluster': 'pixel_meta_cluster',
        'mc_name': 'pixel_meta_cluster_rename'
    },
    axis=1
)

# create the mapping from pixel SOM to pixel meta cluster
pixel_remapped_dict = dict(
    pixel_remapped_data[
        ['pixel_som_cluster', 'pixel_meta_cluster']
    ].values
)

# create the mapping from pixel meta cluster to renamed pixel meta cluster
pixel_renamed_meta_dict = dict(
    pixel_remapped_data[
        ['pixel_meta_cluster', 'pixel_meta_cluster_rename']
    ].drop_duplicates().values
)

# re-compute average channel expression for each pixel meta cluster
# and the number of pixels per meta cluster, add renamed meta cluster column in
print("Re-computing average channel expression across pixel meta clusters")
pixel_channel_avg_meta_cluster = som_utils.compute_pixel_cluster_channel_avg(
    fovs,
    channels,
    base_dir,
    'pixel_meta_cluster',
    pixel_data_dir + '_remapped',
    keep_count=True
)
pixel_channel_avg_meta_cluster['pixel_meta_cluster_rename'] = \
    pixel_channel_avg_meta_cluster['pixel_meta_cluster'].map(pixel_renamed_meta_dict)

# re-save the pixel channel average meta cluster table
pixel_channel_avg_meta_cluster.to_csv(meta_cluster_avg_path, index=False)

# re-assign pixel meta cluster labels back to the pixel channel average som cluster table
print("Re-assigning meta cluster column in pixel SOM cluster average channel expression table")
pixel_channel_avg_som_cluster = pd.read_csv(som_cluster_avg_path)

pixel_channel_avg_som_cluster['pixel_meta_cluster'] = \
    pixel_channel_avg_som_cluster['pixel_som_cluster'].map(pixel_remapped_dict)

pixel_channel_avg_som_cluster['pixel_meta_cluster_rename'] = \
    pixel_channel_avg_som_cluster['pixel_meta_cluster'].map(pixel_renamed_meta_dict)

# re-save the pixel channel average som cluster table
pixel_channel_avg_som_cluster.to_csv(som_cluster_avg_path, index=False)

Re-computing average channel expression across pixel meta clusters
Re-assigning meta cluster column in pixel SOM cluster average channel expression table


Generate the color scheme returned by the interactive reclustering process. This will be for visualizing the pixel cluster overlay.

In [ ]:
raw_cmap, _ = som_utils.generate_meta_cluster_colormap_dict(
    pixel_mcd.output_mapping_filename,
    pixel_mcg.im_cl.cmap
)

### 3.2: pixel cluster overlay (pixel meta clusters only)

In [56]:
# select fovs to display
pixel_fovs = test_fovs[:20]

In [57]:
# define the path to the channel file
if img_sub_folder is None:
    chan_file = os.path.join(
        fovs[0], io_utils.list_files(os.path.join(tiff_dir, fovs[0]), substrs=['.tif', '.tiff'])[0]
    )
else:
    chan_file = os.path.join(
        fovs[0], img_sub_folder, io_utils.list_files(os.path.join(tiff_dir, fovs[0], img_sub_folder), substrs=['.tif', '.tiff'])[0]
    )

# generate the pixel cluster masks for each fov in pixel_fovs
pixel_cluster_masks = data_utils.generate_pixel_cluster_mask(
    pixel_fovs,
    base_dir,
    tiff_dir,
    chan_file=chan_file,
    pixel_data_dir=pixel_data_dir + '_remapped',
    pixel_cluster_col='pixel_meta_cluster'
)

* `save_pixel_masks`: replace with `True` if you want to save, files will be written as `{fov_name}_pixel_mask.tiff` in `pixel_output_dir`

In [58]:
save_pixel_masks = True

if save_pixel_masks:
    data_utils.save_fov_images(
        pixel_fovs,
        os.path.join(base_dir, pixel_output_dir),
        pixel_cluster_masks,
        sub_dir='pixel_masks',
        name_suffix='_pixel_mask'
    )

In [59]:
# mantis creation
create_mantis_project(mantis_project_path=os.path.join(base_dir, pixel_output_dir, 'mantis'),
                      img_data_path=tiff_dir,
                      mask_output_dir=os.path.join(base_dir, pixel_output_dir, 'pixel_masks'),
                      mask_suffix='_pixel_mask',
                      mapping_path = os.path.join(base_dir, pixel_output_dir, pixel_cluster_prefix + '_pixel_meta_cluster_mapping.csv'),
                     seg_dir=segmentation_dir, img_sub_folder='')


In [18]:
io_utils.list_files(os.path.join(base_dir, pixel_output_dir, 'pixel_masks'))

['TONIC_TMA23_R6C1_pixel_mask.tiff',
 'TONIC_TMA15_R11C2_pixel_mask.tiff',
 'TONIC_TMA23_R9C4_pixel_mask.tiff',
 'TONIC_TMA6_R7C4_pixel_mask.tiff',
 'TONIC_TMA23_R6C3_pixel_mask.tiff',
 'TONIC_TMA18_R2C2_pixel_mask.tiff',
 'TONIC_TMA19_R6C6_pixel_mask.tiff',
 'TONIC_TMA18_R9C1_pixel_mask.tiff',
 'TONIC_TMA4_R1C2_pixel_mask.tiff',
 'TONIC_TMA23_R1C2_pixel_mask.tiff']

In [19]:
plot_utils.plot_pixel_cell_cluster_overlay(
    pixel_cluster_masks,
    pixel_fovs,
    os.path.join(base_dir, pixel_meta_cluster_remap_name),
    metacluster_colors=raw_cmap
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [60]:
images = io_utils.list_files(os.path.join(base_dir, pixel_output_dir, 'mantis/TONIC_TMA6_R5C2'), '.tiff')
images = [image.split('.tiff')[0] for image in images]
#images = [image for image in images if image not in channels]
images = [image for image in images if image not in ['ECAD', 'CK17', 'FOXP3', 'CD11c', 'CD56', 'CD20', 'CD8', 'CD3','population_pixel_mask', 'cell_segmentation', 'H3K27me3']]

In [61]:
images

['Au',
 'CD14',
 'CD163',
 'CD31',
 'CD38',
 'CD4',
 'CD45',
 'CD45RB',
 'CD45RO',
 'CD57',
 'CD68',
 'CD69',
 'Calprotectin',
 'ChyTr',
 'Collagen1',
 'FAP',
 'Fe',
 'Fibronectin',
 'GLUT1',
 'H3K9ac',
 'HLA1',
 'HLADR',
 'IDO',
 'Ki67',
 'LAG3',
 'Noodle',
 'PD1',
 'PDL1',
 'SMA',
 'TBET',
 'TCF1',
 'TIM3',
 'Vim',
 'chan_115',
 'chan_141',
 'chan_39',
 'chan_45',
 'chan_48',
 'ECAD_smoothed',
 'CK17_smoothed',
 'CD11c_nuc_exclude',
 'FOXP3_nuc_include']

In [62]:
import shutil
folders = io_utils.list_folders(os.path.join(base_dir, pixel_output_dir, 'mantis'))
for folder in folders:
    for image in images:
        os.remove(os.path.join(base_dir, pixel_output_dir, 'mantis', folder, image + '.tiff'))

### 4: Save parameters for use in cell clustering

The following parameters are saved:

* `fovs`: the subset of fovs
* `channels`: the subset of channels
* `segmentation_dir`: the path to the directory containing your segmentated images per FOV (generated from `Segment_Image_Data.ipynb`)
* `seg_suffix`: the suffix plus the file extension of the segmented images for each FOV
* `pixel_data_dir`: the name of the directory containing tne full pixel data with the pixel SOM and consensus cluster assignments
* `pc_chan_avg_som_cluster_name`: the name of the file containing the average channel expression per pixel SOM cluster
* `pc_chan_avg_meta_cluster_name`: same as above except for pixel meta clusters

The save file will be `{pixel_cluster_prefix}_cell_clustering_params.json` and will be placed in `pixel_output_dir`. Note that the `pixel_output_dir` you use in `example_pixel_clustering.ipynb` should be the same as in `example_cell_clustering.ipynb`.

In [75]:
# define the params dict
cell_clustering_params = {
    'fovs': fovs,
    'channels': channels,
    'segmentation_dir': segmentation_dir,
    'seg_suffix': seg_suffix,
    'pixel_data_dir': pixel_data_dir,
    'pc_chan_avg_som_cluster_name': pc_chan_avg_som_cluster_name,
    'pc_chan_avg_meta_cluster_name': pc_chan_avg_meta_cluster_name
}

# save the params dict
with open(os.path.join(base_dir, pixel_output_dir, '%s_cell_clustering_params.json' % pixel_cluster_prefix), 'w') as fh:
    json.dump(cell_clustering_params, fh)

In [26]:
import shutil
def create_mantis_project(mantis_project_path, img_data_path, mask_output_dir, mask_suffix, mapping_path, 
                          seg_dir, img_sub_folder='normalized'):
    
    if not os.path.exists(mantis_project_path):
        os.makedirs(mantis_project_path)
        
    # create key from cluster number to cluster name
    map_df = pd.read_csv(mapping_path)
    map_df = map_df.loc[:, ['metacluster', 'mc_name']]

    # remove duplicates from df
    map_df = map_df.drop_duplicates()
    map_df = map_df.sort_values(by=['metacluster'])
    
    # rename for mantis names
    map_df = map_df.rename({'metacluster': 'region_id', 'mn_name': 'region_name'}, axis=1)
    
    # get names of fovs with masks
    mask_names = io_utils.list_files(mask_output_dir, mask_suffix)
    fov_names = io_utils.extract_delimited_names(mask_names, delimiter=mask_suffix)
    
    # create a folder with image data, pixel masks, and segmentation mask
    for idx, val in enumerate(fov_names):
        
        # set up paths
        img_source_dir = os.path.join(img_data_path, val, img_sub_folder)
        output_dir = os.path.join(mantis_project_path, val)
        
        # copy image data if not already copied in from previous round of clustering
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

            # copy all channels into new folder
            chans = io_utils.list_files(img_source_dir, '.tiff')
            for chan in chans:
                shutil.copy(os.path.join(img_source_dir, chan), os.path.join(output_dir, chan))

        # copy mask into new folder
        mask_name = mask_names[idx]
        shutil.copy(os.path.join(mask_output_dir, mask_name), os.path.join(output_dir, 'population{}.tiff'.format(mask_suffix)))
        
        # copy segmentations into directory
        seg_name = val + '_feature_0.tif'
        shutil.copy(os.path.join(seg_dir, seg_name), os.path.join(output_dir, 'cell_segmentation.tiff'))
        
        # copy mapping into directory
        map_df.to_csv(os.path.join(output_dir, 'population{}.csv'.format(mask_suffix)), index=False)